In [ ]:
import numpy as np
import pandas as pd
import os
from collections import defaultdict
from shutil import copy
from shutil import copytree, rmtree

import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)

Dataset Loading

In [ ]:
%cd /kaggle/input/food-101/food-101/
os.listdir('./food-101/images')

In [ ]:
os.listdir('./food-101/meta')

Preperation of Data - 

In [ ]:
!head 'food-101/meta/train.txt'

Training Data

In [ ]:
train_df = pd.read_csv('food-101/meta/train.txt', header = None, names=['path'])
train_df.head()

In [ ]:
test_df = pd.read_csv('food-101/meta/test.txt', header = None, names=['path'])
test_df.head()

In [ ]:
def spliter(data, class_or_id='id'):
    if class_or_id.upper() == 'CLASS':
        output = data.split('/')[0]
        
    else:
        output = data.split('/')[-1]
    return output

In [ ]:
train_df['label'] = train_df['path'].map(lambda x: spliter(data = x, class_or_id = 'Class')) 
train_df.head()

In [ ]:
train_df['idx'] = train_df['path'].map(lambda x: spliter(x)) 
train_df.head()

In [ ]:
test_df['label'] = test_df['path'].map(lambda x: spliter(x, 'class')) 
test_df['idx'] = test_df['path'].map(lambda x: spliter(x)) 
test_df.head()

In [ ]:
mini_20 = train_df['label'].unique()[:20]
mini_20 = [f.upper() for f in mini_20]
mini_20

In [ ]:
def prepare_data(label):
    if label.upper() in mini_20:
        return label
    else:
        return 'Others'

In [ ]:
train_df['label'] = train_df['label'].map(lambda x: prepare_data(x))
test_df['label'] = test_df['label'].map(lambda x: prepare_data(x))

In [ ]:
train_df['label'].unique()

In [ ]:
def add_exact_path(path):
    return 'food-101/images/'+path+'.jpg'

In [ ]:
train_df['path'] = train_df[['path']].apply(add_exact_path, axis=1)
test_df['path'] = test_df[['path']].apply(add_exact_path, axis=1)
train_df.head()

In [ ]:
from matplotlib import pyplot as plt
img = plt.imread(train_df['path'].iloc[0])
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
test_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_gen = datagen.flow_from_dataframe(dataframe = train_df, directory=None, x_col='path', y_col='label',
    weight_col=None, target_size=(256, 256), color_mode='rgb',
    classes=None, class_mode='categorical', batch_size=32, shuffle=True)

In [ ]:
test_gen = test_gen.flow_from_dataframe(dataframe = test_df, directory=None, x_col='path', y_col='label',
    weight_col=None, target_size=(256, 256), color_mode='rgb',
    classes=None, class_mode='categorical', batch_size=32, shuffle=True)

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications.xception import Xception

Xception = Xception(weights='imagenet', include_top=False)
x = Xception.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,activation='relu')(x)
# x = Dropout(0.4)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)

predictions = Dense(21, activation='softmax')(x)

model = Model(inputs=Xception.input, outputs=predictions)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit_generator(train_gen,
                              steps_per_epoch = len(train_gen) // 32,
                    epochs=60,
                    verbose=1)

In [ ]:
result = model.evaluate(test_gen, batch_size=32)
print(result)

In [ ]:
model.save('/kaggle/working/food21.hdf5')